In [1]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.support.ui import Select

In [21]:
option = webdriver.ChromeOptions()
option.add_argument(" — incognito")

In [22]:
browser = webdriver.Chrome(executable_path='./chromedriver',
                           chrome_options=option)

In [23]:
wc_url = "https://www.whoscored.com/Regions/247/Tournaments/36/Seasons/3768/Stages/10274/TeamStatistics/International-FIFA-World-Cup-2014"

In [24]:
browser.get(wc_url)

timeout = 20
try:
    WebDriverWait(browser, timeout).until(
        EC.visibility_of_element_located(
            (By.XPATH, "//a[@class='team-link']")))
except TimeoutException:
    print("Timed out waiting for page to load")
    browser.quit()

In [25]:
# Encuentra links de equipos
team_elements = browser.find_elements_by_xpath('//a[@class="team-link"]')
team_links = [(x.text, x.get_attribute('href')) for x in team_elements]
team_links[0]


('Germany', 'https://www.whoscored.com/Teams/336/Show/Germany-Germany')

In [35]:
def get_player_data(browser, id, df, player_name):
    def_element = browser.find_element_by_id(id)
    rows = def_element.find_elements_by_tag_name("tr")    
    header_elements = rows[0].find_elements_by_tag_name('th')
    columns = [x.text for x in header_elements]
    columns.append("Jugador")    
    
    data = []
    for row in rows[1:]:
        value_elements = row.find_elements_by_tag_name('td')
        values = [x.text for x in value_elements]
        values.append(player_name)
        data.append(values)        

    if df is None:
        df = pd.DataFrame(data, columns=columns)
    else:
        df = df.append(pd.DataFrame(data, columns=columns))
        
    return df

def get_players_data(browser):    
    time.sleep(5)
    player_elements = browser.find_elements_by_xpath('//a[@class="player-link"]')
    
    defensive_data = None
    offensive_data = None
    passing_data = None
    
    player_links = [(x.text, x.get_attribute('href')) for x in player_elements]
    for player_name, link in player_links:
        browser.get(link)

        history_element = browser.find_element_by_xpath('//a[text()="History"]')
        history_element.click()

        time.sleep(5)
        stats_element = browser.find_element_by_id('player-tournament-stats-options')        
        stats_options = stats_element.find_elements_by_tag_name('a')

        print("Opciones: ", [x.text for x in stats_options])
        for e in stats_options:
            e.click()
            time.sleep(2)            
            if e.text == "Defensive":                
                defensive_data = get_player_data(browser, 'statistics-table-defensive', defensive_data, player_name)
            if e.text == "Offensive":
                offensive_data = get_player_data(browser, 'statistics-table-offensive', offensive_data, player_name)
            if e.text == "Passing":
                passing_data = get_player_data(browser, 'statistics-table-passing', passing_data, player_name)


        defensive_data.to_csv("Alemania_" + "_def.csv")
        offensive_data.to_csv("Alemania_" + "_off.csv")
        passing_data.to_csv("Alemania_" + "_pass.csv")

        browser.back()
        browser.back()
    
    return defensive_data, offensive_data, passing_data

In [36]:
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

for team, link in team_links:
    link = link.replace("Show", "Archive")
    browser.get(link)
    
    seasons = Select(browser.find_element_by_id("stageId"))
    for e in seasons.options:
        if e.text == "FIFA World Cup - 2014":            
            e.click()            
            # Get player attributes
            players_data = get_players_data(browser)
            break
            
    time.sleep(5)
    break
    
players_data[0]

Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']
Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']
Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']
Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']
Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']
Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']
Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']
Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']
Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']
Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']
Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']
Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']
Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']


IndexError: list index out of range

In [14]:
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

for team, link in team_links:
    link = link.replace("Show", "Archive")
    browser.get(link)
    
    seasons = Select(browser.find_element_by_id("stageId"))
    for e in seasons.options:
        if e.text == "FIFA World Cup - 2014":            
            e.click()            
            # Get player attributes
            players_data = get_players_data(browser)
            break
            
    time.sleep(5)
    break
    
players_data[0]

Opciones:  ['Summary', 'Defensive', 'Offensive', 'Passing', 'Detailed']


,Season,Team,Tournament,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Jugador
0,2017/2018,Bayern Munich,GB,20(7),1824,1.2,0.3,0.8,-,0.4,0.5,-,-,7.28,Thomas Müller
1,2017/2018,Bayern Munich,UCL,7(2),644,1,0.3,1.6,-,0.3,0.4,-,-,7.02,Thomas Müller
2,2016/2017,Bayern Munich,GB,25(4),2265,0.5,0.7,0.7,0.1,0.3,0.6,0.2,-,7.30,Thomas Müller
3,2016/2017,Bayern Munich,UCL,5(4),463,0.6,0.3,0.4,0.1,0.3,0.3,0.1,-,6.72,Thomas Müller
4,2015/2016,Bayern Munich,GB,26(5),2345,0.8,0.5,0.8,-,0.5,0.6,-,-,7.50,Thomas Müller
5,2015/2016,Bayern Munich,UCL,9(3),926,0.7,0.4,0.8,-,0.1,1.1,0.2,-,7.19,Thomas Müller
6,2014/2015,Bayern Munich,GB,28(4),2374,0.7,0.4,0.9,-,0.2,0.8,0.1,-,7.27,Thomas Müller
7,2014/2015,Bayern Munich,UCL,10,778,0.9,0.5,1.1,-,0.7,1.6,0.1,-,7.59,Thomas Müller
8,2016,Germany,UEC,6,570,2.7,0.5,1.5,0.2,1.2,1.7,-,-,7.24,Thomas Müller
9,2013/2014,Bayern Munich,UCL,8(4),710,0.4,0.7,0.9,-,0.5,0.4,0.1,-,6.89,Thomas Müller


In [ ]:
# find_elements_by_xpath returns an array of selenium objects.

titles_element = browser.find_elements_by_xpath(“//a[@class=’text-bold’]”)
# use list comprehension to get the actual repo titles and not the selenium objects.
titles = [x.text for x in titles_element]
# print out all the titles.
print('titles:')
print(titles, '\n')

In [2]:
wc_url = "https://www.whoscored.com/Regions/247/Tournaments/36/Seasons/3768/Stages/10274/TeamStatistics/International-FIFA-World-Cup-2014"

In [3]:
wc_text = get_url_text(wc_url)


In [4]:
strips = re.findall('class="team-link"(.*?)</td>', wc_text)

[]

In [5]:
wc_text

'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n\r\n<!--[if IE 7 ]>\r\n<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" class="cmePineapple  no-js msie msie-7 cme-ie cme-ie7 cme-lte7 cme-lte8 cme-lte9">\r\n<link rel="stylesheet" href="/etc/clientlibs/cmegroup/cmegroupClientLibs/css/cmePrint.css" type="text/css" media="print" />\r\n<![endif]-->\r\n<!--[if IE 8 ]>\r\n<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" class="cmePineapple  no-js msie msie-8 cme-ie cme-ie8 cme-lte8 cme-lte9">\r\n<link rel="stylesheet" href="/etc/clientlibs/cmegroup/cmegroupClientLibs/css/cmePrint.css" type="text/css" media="print" />\r\n<![endif]-->\r\n<!--[if IE 9]>\r\n<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml" class="cmePineapple  no-js msie msie-9 cme-ie cme-ie9 cme-lte9">\r\n<![endif]-->\r\n<!--[if (gt IE 9) |!(IE)]><!-->\r\n<html lan